In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import kagglehub

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download Dataset
path = kagglehub.dataset_download("andrewmvd/medical-mnist")

print("Path to dataset files:", path)

Path to dataset files: /Users/allenavila/.cache/kagglehub/datasets/andrewmvd/medical-mnist/versions/1


In [3]:
# Create dataset
BATCHSIZE = 64
IMG_HEIGHT = 48
IMG_WIDTH = 48

train_ds = keras.utils.image_dataset_from_directory(
    path,
    validation_split = 0.2,
    subset = "training",
    seed = 123,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCHSIZE
)

validation_ds = keras.utils.image_dataset_from_directory(
    path,
    validation_split = 0.2,
    subset = "validation",
    seed = 123,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCHSIZE
)

Found 58954 files belonging to 6 classes.
Using 47164 files for training.
Found 58954 files belonging to 6 classes.
Using 11790 files for validation.


In [4]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [5]:
class_names = train_ds.class_names
class_names

['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']

In [6]:
# convert train and validation datasets into NumPy arrays
def dataset_to_numpy(dataset):
    images = []
    labels = []

    for image_batch, label_batch in dataset:
        images.append(image_batch.numpy())
        labels.append(label_batch.numpy())

    images = np.concatenate(images, axis=0)
    labels = np.concatenate(images, axis=0)

    return images, labels

In [ ]:
X_train, y_train = dataset_to_numpy(train_ds)
X_valid, y_valid = dataset_to_numpy(validation_ds)

2024-12-12 13:25:59.617344: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# Create Model
# CNN Model
# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D())

# model.add(keras.layers.Flatten())

# model.add(keras.layers.Dense())
# model.add(keras.layers.Dense(units = 6, activation="softmax"))